In [13]:
import numpy as np
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx
array = np.linspace(-2.0,2.0,100)
print(array)

value = -20.0

print(find_nearest(array, value))

[-2.         -1.95959596 -1.91919192 -1.87878788 -1.83838384 -1.7979798
 -1.75757576 -1.71717172 -1.67676768 -1.63636364 -1.5959596  -1.55555556
 -1.51515152 -1.47474747 -1.43434343 -1.39393939 -1.35353535 -1.31313131
 -1.27272727 -1.23232323 -1.19191919 -1.15151515 -1.11111111 -1.07070707
 -1.03030303 -0.98989899 -0.94949495 -0.90909091 -0.86868687 -0.82828283
 -0.78787879 -0.74747475 -0.70707071 -0.66666667 -0.62626263 -0.58585859
 -0.54545455 -0.50505051 -0.46464646 -0.42424242 -0.38383838 -0.34343434
 -0.3030303  -0.26262626 -0.22222222 -0.18181818 -0.14141414 -0.1010101
 -0.06060606 -0.02020202  0.02020202  0.06060606  0.1010101   0.14141414
  0.18181818  0.22222222  0.26262626  0.3030303   0.34343434  0.38383838
  0.42424242  0.46464646  0.50505051  0.54545455  0.58585859  0.62626263
  0.66666667  0.70707071  0.74747475  0.78787879  0.82828283  0.86868687
  0.90909091  0.94949495  0.98989899  1.03030303  1.07070707  1.11111111
  1.15151515  1.19191919  1.23232323  1.27272727  1.3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
import math,cmath
from scipy.sparse import diags
from scipy.integrate import quad
from mpmath import *
from scipy import integrate
from scipy import optimize
from scipy.misc import derivative

Nmst = 201; #Number of lattice points
Pbst = 101;#Probe attachment site
lbd = 0.0 ;#Lambda strength ofAAH
irrb = (1 + np.sqrt(5))/2
t = 1.0; # hopping potential for sites
to = 3.0; # hopping potential for bath
sitegammaindx = [0, Nmst-1, Pbst-1]
sitegammastrn = [1.0, 1.0, 0.1]
siteindx = np.array(range(1, Nmst+1))
sitepotential = 2*lbd*np.cos(2*np.pi*irrb*(siteindx))
diagonals = [sitepotential,t*np.ones(Nmst-1), t*np.ones(Nmst-1)]
offset = [0,-1,1]
sys_Ham = diags(diagonals,offset,dtype='complex_').toarray()
del_mu = 2.0
mu_L = 0.2 - del_mu
mu_p = 0.2
mu_R = 0.2 + del_mu
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx
def selfenergy(gamma,site,energy):#self energy matrix
    mat = np.zeros((Nmst,Nmst),dtype = 'complex_')
    mat[site,site] =  ((gamma**2/(2*to**2))*(energy - cmath.sqrt(4*to**2-energy**2)*1j))
    return mat
def specden(gamma,energy):#spectral density matrix(-2Im(sigma))
    mat =  -2*((gamma**2/(2*to**2))*(energy - cmath.sqrt(4*to**2-energy**2)*1j)).imag
    return mat
def Green_func(energy):#green's function
    mat = energy*np.identity(Nmst) - sys_Ham - selfenergy(sitegammastrn[0],sitegammaindx[0],energy) - \
    selfenergy(sitegammastrn[1],sitegammaindx[1],energy) - selfenergy(sitegammastrn[2],sitegammaindx[2],energy)
    return (np.linalg.det(mat)/t)
def advan_gren(energy):
    mat = np.transpose(np.conjugate(Green_func(energy)))
    return mat
def transmissionprob(sitstrn1, sitstrn2, energy):
    spcdn1 = specden(sitstrn1, energy)
    retgre = Green_func(energy)
    spcdn2 = specden(sitstrn2, energy)
    mat = (spcdn1*spcdn2)/(abs(retgre)**2)
    return mat
point = np.linspace(-4.0,4.0, 3000)
transmissionprob = np.vectorize(transmissionprob)
first_trans_probe = transmissionprob(sitegammastrn[2], sitegammastrn[0],point)
second_trans_probe = transmissionprob(sitegammastrn[2], sitegammastrn[1],point)
first_trans_right = transmissionprob(sitegammastrn[1], sitegammastrn[0],point)
second_trans_right = transmissionprob(sitegammastrn[1], sitegammastrn[2],point)
def current_vallp(mu_p,mu_l):#integral equation
    probe_energy = find_nearest(point,mu_p)
    left_energy = find_nearest(point,mu_l)
    if left_energy == probe_energy:
        print('error')
    if left_energy > probe_energy:
        result = first_trans_probe[probe_energy:left_energy]
    else:
        result = first_trans_probe[left_energy:probe_energy]
    return result
def current_valpr(mu_p,mu_r):
    probe_energy = find_nearest(point,mu_p)
    right_energy = find_nearest(point,mu_r) 
    if right_energy == probe_energy:
        print('error')
    if right_energy < probe_energy:
        result = second_trans_probe[right_energy:probe_energy]
    else:
        result = second_trans_probe[probe_energy:right_energy]
    return result
def current_int(mu_p,mu_l,mu_r):
    probe_energy = find_nearest(point,mu_p)
    right_energy = find_nearest(point,mu_r)
    left_energy = find_nearest(point,mu_l)
    if left_energy == probe_energy:
        prime = 0.0
    elif probe_energy < left_energy:
        point1 = point[probe_energy:left_energy]
        current_val20 = np.vectorize(current_vallp)
        I1 = current_val20(mu_p,mu_l)
        prime = integrate.simps(I1,point1)
    else:
        point1 = point[left_energy:probe_energy]
        current_val20 = np.vectorize(current_vallp)
        I1 = current_val20(mu_p,mu_l)
        prime = integrate.simps(I1,point1)
    if right_energy == probe_energy:
        primeo = 0.0
    elif probe_energy < left_energy:
        point1 = point[probe_energy:left_energy]
        current_val20 = np.vectorize(current_vallp)
        I1 = current_val20(mu_p,mu_l)
        primeo = integrate.simps(I1,point1)
    else:
        point2 = point[probe_energy:right_energy]
        current_val30 = np.vectorize(current_valpr)
        I2 = current_val30(mu_p,mu_r)
        primeo = integrate.simps(I2,point2)
    return (prime + primeo)
def deriv_current(mu_p,mu_l,mu_r) :
    return derivative(lambda x: current_int(x,mu_l,mu_r),mu_p)
def min_probe_potential(mu_l,mu_r):#Newton-Raphson minimization
    Mu = (mu_r+mu_l)/2
    current =  current_int(Mu,mu_l,mu_r)
    while abs(current) >= 1.0e-10:
        Mu = Mu - (current_int(Mu,mu_l,mu_r)/(deriv_current(Mu,mu_l,mu_r)))
        current =  current_int(Mu,mu_l,mu_r)
    return Mu
print(min_probe_potential(mu_L,mu_R))
def current_fulllr(mu_l, mu_r):
    right_energy = find_nearest(point,mu_r)
    left_energy = find_nearest(point,mu_l)  
    print(first_trans_probe[left_energy:right_energy],'l')
    return first_trans_probe[left_energy:right_energy]
def current_fulllp(mu_r, mu_l):
    mu_poi = min_probe_potential(mu_l,mu_r)
    right_energy = find_nearest(point,mu_r)
    probe_energy = find_nearest(point,mu_poi)  
    print(first_trans_probe[probe_energy:right_energy],'m')
    return first_trans_probe[probe_energy:right_energy]
def current_full2(mu_l,mu_r):  
    mu_poi = min_probe_potential(mu_l,mu_r)
    probe_energy = find_nearest(point,mu_poi)
    right_energy = find_nearest(point,mu_r)
    left_energy = find_nearest(point,mu_l)
    point1 = point[left_energy:right_energy]
    current_val20 = np.vectorize(current_fulllr)
    I1 = current_val20(mu_l,mu_r)
    point2 = point[probe_energy:right_energy]
    current_val30 = np.vectorize(current_fulllp)
    I2 = current_val30(mu_poi,mu_r)
    if left_energy == right_energy:
        prime = 0.0
    else:
        prime = integrate.simps(I1,point1)
    if right_energy == probe_energy:
        primeo = 0.0
    else:
        primeo = integrate.simps(I2,point2)
    return (prime + primeo)

/tmp/ipykernel_87196/3004419233.py:114: RuntimeWarning: divide by zero encountered in double_scalars
  Mu = Mu - (current_int(Mu,mu_l,mu_r)/(deriv_current(Mu,mu_l,mu_r)))


j
[]
None


"\ndef current_int(mu_p,mu_l,mu_r):\n    probe_energy = find_nearest(point,mu_p)\n    right_energy = find_nearest(point,mu_r)\n    left_energy = find_nearest(point,mu_l)\n    point1 = point[left_energy:probe_energy]\n    current_val20 = np.vectorize(current_vallp)\n    I1 = current_val20(mu_p,mu_l)\n    point2 = point[probe_energy:right_energy]\n    current_val30 = np.vectorize(current_valpr)\n    I2 = current_val30(mu_p,mu_r)\n    return(integrate.simps(I1,point1) + integrate.simps(I2,point2))\ndef deriv_current(mu_p,mu_l,mu_r) :\n    return derivative(lambda x: current_int(x,mu_l,mu_r),mu_p)\ndef min_probe_potential(mu_l,mu_r):#Newton-Raphson minimization\n    Mu = (mu_r+mu_l)/2\n    current =  current_int(Mu,mu_l,mu_r)\n    while abs(current) >= 1.0e-10:\n        Mu = Mu - (current_int(Mu,mu_l,mu_r)/(deriv_current(Mu,mu_l,mu_r)))\n        current =  current_int(Mu,mu_l,mu_r)\n    return Mu\ndef current_fulllr(mu_l, mu_r):\n    right_energy = find_nearest(point,mu_r)\n    left_energy